In [1]:
import requests

In [2]:
data = {
    "name": "post test from python",
    "day": "26",
    "film": "Oppenheimer",
    "released": "2023",
    "rating": "0",
}

In [6]:
r = requests.post("http://127.0.0.1:8000/add/", data=data)

In [7]:
r.content

b'{"id":5,"name":"post test from python","day":"26","film":"Oppenheimer","released":"2023","rating":"0","created":"2023-11-27T18:30:09.860377Z"}'

In [2]:
r = requests.get("http://127.0.0.1:8000/endpoint/user_table/")

In [6]:
df = pd.DataFrame(json.loads(r.content))

NameError: name 'json' is not defined

In [ ]:
# Let's try to scrape for a user and enter it in

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pandas as pd
import json
print(pd.__version__)

# Extract info username

user = "trschwab"
BASE_URL = "https://letterboxd.com"

2.0.1


In [20]:
def get_diary(a_user):
    return f"{BASE_URL}/{a_user}/films/diary/"

In [23]:
diary_url = get_diary(user)
r = requests.get(diary_url)
soup = BeautifulSoup(r.content)
table = soup.find_all('table')
df = pd.read_html(str(table), extract_links="all")

In [24]:
df

[                                     (Month, None)   
 0   (Nov 2023, /trschwab/films/diary/for/2023/11/)  \
 1                                         (, None)   
 2                                         (, None)   
 3                                         (, None)   
 4                                         (, None)   
 5                                         (, None)   
 6                                         (, None)   
 7                                         (, None)   
 8                                         (, None)   
 9   (Oct 2023, /trschwab/films/diary/for/2023/10/)   
 10                                        (, None)   
 11                                        (, None)   
 12  (Sep 2023, /trschwab/films/diary/for/2023/09/)   
 13                                        (, None)   
 14                                        (, None)   
 15  (Aug 2023, /trschwab/films/diary/for/2023/08/)   
 16                                        (, None)   
 17       

In [ ]:
# Test 2:

In [25]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from PIL import Image, ImageEnhance
import urllib.request
from io import BytesIO
import numpy as np

BASE_URL = "https://letterboxd.com"

def generate_topster(user):
    # Generate topster for user

    user_df = get_user_diary_info(user)

    c_names = ["month", "day", "film", "released", "rating", "like", "rewatch", "review", "edityou", "film_url"]  # , "img_url", "small_img_url"]

    for i in range(len(c_names)):
        user_df.rename(columns={ user_df.columns[i]: c_names[i] }, inplace=True)

    # Sort by rating
    user_df = user_df.sort_values("rating", ascending=False).drop_duplicates('film_url', keep="first")

    # Sort by rating
    # user_df = user_df.sort_values("rating", ascending=False)

    # delete duplicates
    # user_df.drop_duplicates(subset='film', keep="last", inplace=True)


    user_df = user_df.head(25)

    user_df["img_url"] = user_df["film_url"].apply(lambda x: add_img(x))
    user_df["small_img_url"] = user_df["film_url"].apply(lambda x: add_small_img(x))

    head_test = user_df

    a_list = list(head_test["small_img_url"])

    b_list = np.reshape(a_list, (-1, 5))

    new = Image.new("RGBA", (1150,1725))
    for row in range(len(b_list)):
        for col in range(len(b_list[row])):
            try:
                response = requests.get(b_list[row][col])
                img = Image.open(BytesIO(response.content))
                img = img.resize((230,345))
                new.paste(img, (col * 230, row * 345))
            except Exception as e:
                print(e)
    new.save(f"{user}.png")


def add_img(x):
    r = requests.get(x)
    soup = BeautifulSoup(r.content)
    img = soup.find("meta", property="og:image")
    return img["content"]

def add_small_img(x):
    try:
        r = requests.get(x)
        soup = BeautifulSoup(r.content)
        s = soup.find("script", attrs={"type": "application/ld+json"})
        split_str = s.string.split("/* <![CDATA[ */")[-1].split("/* ]]> */")[0]
        info = json.loads(split_str)
        return info["image"]
    except Exception as e:
        print(e)
        return None

def gen_film_url(a_str):
    return f"https://letterboxd.com/film/{a_str[1].split('/')[3]}/"


def get_diary(a_user):
    return f"{BASE_URL}/{a_user}/films/diary/"

def get_user_diary_info(a_user):
    '''
    returns a DF of a user's diary information

    # TOOD need to be able to handle users that do not exist
    '''
    df_list = []
    # Get diary URL
    diary_url = get_diary(a_user)
    print(diary_url)
    while diary_url:
        # print(diary_url)
        r = requests.get(diary_url)
        soup = BeautifulSoup(r.content)

        # Extract the table as a df
        table = soup.find_all('table')
        df_list.append(pd.read_html(str(table), extract_links="all")[0])

        # Find out if there are any subsequent pages
        older_link = soup.find_all("a", text="Older")

        if older_link:
            new_page = older_link[0]['href']
            diary_url = f"{BASE_URL}{new_page}"
        else:
            diary_url = False
    return_df = pd.concat(df_list)

    return_df["film_url"] = return_df[('Film', None)].apply(lambda x: gen_film_url(x))

    return return_df


In [68]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from PIL import Image, ImageEnhance
import urllib.request
from io import BytesIO
import numpy as np

BASE_URL = "https://letterboxd.com"
# grahamgearhart
def post_user_into(user):
    # Generate topster for user

    user_df = get_user_diary_info(user)

    c_names = ["month", "day", "film", "released", "rating", "like", "rewatch", "review", "edityou", "film_url"]  # , "img_url", "small_img_url"]

    for i in range(len(c_names)):
        user_df.rename(columns={ user_df.columns[i]: c_names[i] }, inplace=True)

    # explode tuple values into new cols:

    user_df[["month", "month_none"]] = pd.DataFrame(user_df['month'].tolist(), index=user_df.index)
    user_df[["day", "day_none"]] = pd.DataFrame(user_df['day'].tolist(), index=user_df.index)
    user_df[["film", "film_link"]] = pd.DataFrame(user_df['film'].tolist(), index=user_df.index)
    user_df[["released", "released_none"]] = pd.DataFrame(user_df['released'].tolist(), index=user_df.index)
    user_df[["rating", "rating_none"]] = pd.DataFrame(user_df['rating'].tolist(), index=user_df.index)
    user_df[["review", "review_link"]] = pd.DataFrame(user_df['review'].tolist(), index=user_df.index)

    # Convert to dict for iterative processing.. TODO fix this
    dict_df = user_df.to_dict('records')

    for count, item in enumerate(dict_df):
        if item["month"] == '':
            item["month"] = dict_df[count-1]["month"]

    dated_df = pd.DataFrame(dict_df)
    dated_df[["month", "year"]] = dated_df['month'].str.split(' ', expand=True)

    dated_df_dict = dated_df.to_dict('records')

    # Issue posts
    for item in dated_df_dict:
        data = {
        "name": user, 
        "day": item["day"],
        "month": item["month"],
        "year": item["year"],
        "film": item["film"],
        "released": item["released"],
        "rating":  item["rating"],
        "review_link": item["review_link"],
        "film_link" : item["film_link"]
        }

        r = requests.post("http://127.0.0.1:8000/add/", data=data)
        print(r)
        print(r.content)

    return dated_df

In [76]:
post_user_into("coose")

https://letterboxd.com/coose/films/diary/
<Response [200]>
b'{"id":1352,"name":"coose","day":"25","month":"Nov","year":"2023","film":"No Hard Feelings","released":"2023","rating":"\xc3\x97 \xe2\x98\x85\xe2\x98\x85\xe2\x98\x85","review_link":"/coose/film/no-hard-feelings-2023/","film_link":"/coose/film/no-hard-feelings-2023/","created":"2023-11-27T22:49:32.422286Z"}'
<Response [200]>
b'{"id":1353,"name":"coose","day":"25","month":"Nov","year":"2023","film":"Maggie Moore(s)","released":"2023","rating":"\xc3\x97 \xe2\x98\x85\xe2\x98\x85\xc2\xbd","review_link":"/coose/film/maggie-moores/","film_link":"/coose/film/maggie-moores/","created":"2023-11-27T22:49:32.432965Z"}'
<Response [200]>
b'{"id":1354,"name":"coose","day":"25","month":"Nov","year":"2023","film":"Extraction 2","released":"2023","rating":"\xc3\x97 \xe2\x98\x85\xe2\x98\x85\xe2\x98\x85\xc2\xbd","review_link":"/coose/film/extraction-2/","film_link":"/coose/film/extraction-2/","created":"2023-11-27T22:49:32.443666Z"}'
<Response [2

,month,day,film,released,rating,like,rewatch,review,edityou,film_url,month_none,day_none,film_link,released_none,rating_none,review_link,year
0,Nov,25,No Hard Feelings,2023,× ★★★,"(, None)","(, None)",Read the review,"(Edit this entry, #)",https://letterboxd.com/film/no-hard-feelings-2...,/coose/films/diary/for/2023/11/,/coose/films/diary/for/2023/11/25/,/coose/film/no-hard-feelings-2023/,None,#,/coose/film/no-hard-feelings-2023/,2023
1,Nov,25,Maggie Moore(s),2023,× ★★½,"(, None)","(, None)",Read the review,"(Edit this entry, #)",https://letterboxd.com/film/maggie-moores/,None,/coose/films/diary/for/2023/11/25/,/coose/film/maggie-moores/,None,#,/coose/film/maggie-moores/,2023
2,Nov,25,Extraction 2,2023,× ★★★½,"(, None)","(, None)",Read the review,"(Edit this entry, #)",https://letterboxd.com/film/extraction-2/,None,/coose/films/diary/for/2023/11/25/,/coose/film/extraction-2/,None,#,/coose/film/extraction-2/,2023
3,Nov,24,Asteroid City,2023,× ★★★★½,"(, None)","(, None)",Read the review,"(Edit this entry, #)",https://letterboxd.com/film/asteroid-city/,None,/coose/films/diary/for/2023/11/24/,/coose/film/asteroid-city/,None,#,/coose/film/asteroid-city/,2023
4,Nov,24,The Blackening,2022,× ★★★,"(, None)","(, None)",Read the review,"(Edit this entry, #)",https://letterboxd.com/film/the-blackening/,None,/coose/films/diary/for/2023/11/24/,/coose/film/the-blackening/,None,#,/coose/film/the-blackening/,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,Jun,10,Mayor,2020,× ★★★½,"(, None)","(, None)",Read the review,"(Edit this entry, #)",https://letterboxd.com/film/mayor/,None,/coose/films/diary/for/2021/06/10/,/coose/film/mayor/,None,#,/coose/film/mayor/,2021
840,Jun,08,The Old Guard,2020,× ★★,"(, None)","(, None)",Read the review,"(Edit this entry, #)",https://letterboxd.com/film/the-old-guard-2020/,None,/coose/films/diary/for/2021/06/08/,/coose/film/the-old-guard-2020/,None,#,/coose/film/the-old-guard-2020/,2021
841,Jun,07,Roma,1972,× ★★★★½,"(, None)","(, None)",Read the review,"(Edit this entry, #)",https://letterboxd.com/film/roma/,None,/coose/films/diary/for/2021/06/07/,/coose/film/roma/,None,#,/coose/film/roma/,2021
842,Jun,06,The Clowns,1970,× ★★½,"(, None)","(, None)",Read the review,"(Edit this entry, #)",https://letterboxd.com/film/the-clowns/,None,/coose/films/diary/for/2021/06/06/,/coose/film/the-clowns/,None,#,/coose/film/the-clowns/,2021


In [67]:
dated_df

,month,day,film,released,rating,like,rewatch,review,edityou,film_url,month_none,day_none,film_link,released_none,rating_none,review_link,year
0,Nov,26,Duel,1971,× ★★★★,"(, None)","(, None)",,"(Edit this entry, #)",https://letterboxd.com/film/duel/,/grahamgearhart/films/diary/for/2023/11/,/grahamgearhart/films/diary/for/2023/11/26/,/grahamgearhart/film/duel/,None,#,None,2023
1,Nov,25,Napoleon,2023,× ★★★★,"(, None)","(, None)",Read the review,"(Edit this entry, #)",https://letterboxd.com/film/napoleon-2023/,None,/grahamgearhart/films/diary/for/2023/11/25/,/grahamgearhart/film/napoleon-2023/,None,#,/grahamgearhart/film/napoleon-2023/,2023
2,Nov,23,Prometheus,2012,× ★★★★,"(, None)","(, None)",,"(Edit this entry, #)",https://letterboxd.com/film/prometheus/,None,/grahamgearhart/films/diary/for/2023/11/23/,/grahamgearhart/film/prometheus/,None,#,None,2023
3,Nov,22,Speed,1994,× ★★★★,"(, None)","(, None)",Read prior review,"(Edit this entry, #)",https://letterboxd.com/film/speed/,None,/grahamgearhart/films/diary/for/2023/11/22/,/grahamgearhart/film/speed/2/,None,#,/grahamgearhart/film/speed/2/,2023
4,Nov,22,Spotlight,2015,× ★★★★★,"(, None)","(, None)",Read the review,"(Edit this entry, #)",https://letterboxd.com/film/spotlight/,None,/grahamgearhart/films/diary/for/2023/11/22/,/grahamgearhart/film/spotlight/1/,None,#,/grahamgearhart/film/spotlight/1/,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1218,Feb,08,Snowpiercer,2013,× ★★★★,"(, None)","(, None)",Read prior review,"(Edit this entry, #)",https://letterboxd.com/film/snowpiercer/,/grahamgearhart/films/diary/for/2020/02/,/grahamgearhart/films/diary/for/2020/02/08/,/grahamgearhart/film/snowpiercer/,None,#,/grahamgearhart/film/snowpiercer/,2020
1219,Feb,07,Parasite,2019,× ★★★★★,"(, None)","(, None)",Read prior review,"(Edit this entry, #)",https://letterboxd.com/film/parasite-2019/,None,/grahamgearhart/films/diary/for/2020/02/07/,/grahamgearhart/film/parasite-2019/1/,None,#,/grahamgearhart/film/parasite-2019/1/,2020
1220,Jan,19,The Young Pope,2016,× ★★★★½,"(, None)","(, None)",Read the review,"(Edit this entry, #)",https://letterboxd.com/film/the-young-pope/,/grahamgearhart/films/diary/for/2020/01/,/grahamgearhart/films/diary/for/2020/01/19/,/grahamgearhart/film/the-young-pope/,None,#,/grahamgearhart/film/the-young-pope/,2020
1221,Dec,12,Tropic Thunder: Rain of Madness,2008,× ★★★★★,"(, None)","(, None)",,"(Edit this entry, #)",https://letterboxd.com/film/tropic-thunder-rai...,/grahamgearhart/films/diary/for/2019/12/,/grahamgearhart/films/diary/for/2019/12/12/,/grahamgearhart/film/tropic-thunder-rain-of-ma...,None,#,None,2019


In [77]:
r = requests.get("http://127.0.0.1:8000/")

In [80]:
get_test_df = pd.DataFrame(json.loads(r.content))

In [82]:
get_test_df.to_csv("test_df.csv")